# CAP Accounting and Management

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    tbl_results = soup.find('table', {'class': 'listing'})
    a_links = []
    for a in tbl_results.find_all('a'):
           if 'Resumo' in a.text:
                a_links.append(a['href'])
    return a_links



In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links



In [5]:
url_search_text = 'http://revistas.utfpr.edu.br/pb/index.php/CAP/search/search?query=__query__&searchJournal=11&authors=&title=&abstract=&galleyFullText=&suppFiles=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=&orderDir=&searchPage=__pg_num__#results'

links = listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)

link = links[0]

link

'http://revistas.utfpr.edu.br/pb/index.php/CAP/article/view/2108'

In [6]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [8]:
def captura_titulo(soup):
    try:
        title_tag = soup.find('div', {'id': 'articleTitle'})
        title = title_tag.find('h3').text
    except:
        title = ''
    return {'Título': title}

In [9]:
captura_titulo(soup)['Título']

'Análise dos artigos sobre a contabilidade ambiental nos periódicos de contabilidade'

In [10]:
def captura_data(soup):
    try:
        year_tags = soup.find('div', {'id': 'breadcrumb'})
        year_tag = year_tags.find_all('a')
        year_text = year_tag[1].text
        year = year_text[-5:-1]
    except:
        year = ''
    return {'Data de Publicação': year}

In [11]:
captura_data(soup)

{'Data de Publicação': '2016'}

In [12]:
def captura_resumo(soup):
    try:
        abstract_tag = soup.find('div', {'id': 'articleAbstract'})
        abstract = abstract_tag.find('div').text
    except:
        abstract = ''
    return {'Resumo': abstract}

In [13]:
captura_resumo(soup)

{'Resumo': 'A crise ambiental está diretamente vinculada ŕs açőes humanas, visto que o ser humano no decorrer de sua evoluçăo cria e aprimora ferramentas que permitem a utilizaçăo dos recursos naturais de forma mais rápida e devastadora. A cadeia de produçăo é um sistema complexo, que impulsiona a degradaçăo ambiental, criada pelo desenvolvimento econômico, que foi alavancado pelo capital particular de grandes organizaçőes. As organizaçőes săo atores importantes nessa crise, pois săo responsáveis pela produçăo de bens de consumo para a sociedade e essa produçăo muitas vezes năo se preocupa com o meio ambiente. Diante desse cenário, é visto uma crescente preocupaçăo da sociedade em geral, principalmente pelo Estado, por questőes ambientais. A contabilidade ambiental surge como ferramenta de acompanhamento e controle sobre as açőes das organizaçőes em relaçăo a suas açőes com o meio ambiente. Devida a importância da contabilidade nesse contexto, este estudo, tem o objetivo de verificar a

In [14]:
def captura_palavras_chave(soup):
    try:
        kw_tag = soup.find('div', {'id': 'articleSubject'})
        kw = kw_tag.find('div').text.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    return {'Palavras-chave': kw}

In [15]:
captura_palavras_chave(soup)

{'Palavras-chave': ''}

In [16]:
def captura_autores(soup):
    autores = []
    try:
        authors_tag = soup.find('div', {'id': 'authorString'})
        authors_full = authors_tag.find('em')
        autores = authors_full.text.split(', ')
    except:
        pass

    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [17]:
authors_tag = soup.find('div', {'id': 'authorString'})
authors_full = authors_tag.find('em')
autores = authors_full.text.split(', ')

autores

['Cleber Broietti']

In [18]:
captura_autores(soup)

{'Qtd. de Autores': 1,
 'Autor 1': 'Cleber Broietti',
 'Autor 2': '',
 'Autor 3': '',
 'Autor 4': '',
 'Autor 5': '',
 'Autor 6': ''}

In [19]:
captura_palavras_chave(soup)

{'Palavras-chave': ''}

In [20]:
def captura_afiliacao(soup):
    affiliation_list = []
    try:
        authors_tag = soup.find_all('div', {'class': 'authorBio'})
        for p in authors_tag:
            try:
                affiliation = p.find_all('br')[-2].next_sibling
                affiliation = affiliation.replace('\t', '')
            except:
                affiliation = ''
            affiliation_list.append(affiliation)
    except:
        pass
    affiliation_list = completar_lista(affiliation_list)
    key_affiliation = [f"Afilição {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, affiliation_list))

In [21]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'CAP Accounting and Management'}
    url_search_text = 'http://revistas.utfpr.edu.br/pb/index.php/CAP/search/search?query=__query__&searchJournal=11&authors=&title=&abstract=&galleyFullText=&suppFiles=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=&orderDir=&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        if 'Pré-textual' in captura_titulo(soup)['Título'] or 'CAP Accounting and Management' in captura_titulo(soup)['Título']:
            continue
            
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y')
    return df

## Pesquisa por artigos

In [22]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head()

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afilição 1,Afilição 2,Afilição 3,Afilição 4,Afilição 5,Afilição 6,Url Artigo,Revista
0,2016-01-01,Análise dos artigos sobre a contabilidade ambi...,A crise ambiental está diretamente vinculada ŕ...,,1,Cleber Broietti,,,,,,,,,,,,http://revistas.utfpr.edu.br/pb/index.php/CAP/...,CAP Accounting and Management
1,2015-01-01,Empresas sustentáveis no Brasil: o que elas te...,Empresas sustentáveis săo aquelas que atuam em...,,2,Cintia Maria Araújo,Leonardo José Seixas Pinto,,,,,,,,,,,http://revistas.utfpr.edu.br/pb/index.php/CAP/...,CAP Accounting and Management
2,2020-01-01,INVESTIGAÇĂO DOS ARTEFATOS GERENCIAIS UTILIZAD...,O presente estudo objetiva investigar os artef...,,5,Rosa Jaqueline Ortiz Hamermüller,Yuri Gomes Paiva Azevedo,Hellen Bomfim Gomes,Raimundo Marciano de Freitas Neto,Aneide Oliveira Araújo,,,,,,,,http://revistas.utfpr.edu.br/pb/index.php/CAP/...,CAP Accounting and Management
3,2017-01-01,CONTABILIDADE DE CUSTOS SOB A ÓTICA DA TEORIA ...,A perspectiva institucional está se tornando u...,,3,Keila Priscila Santos,Juliano Francisco Baldissera,Denis Dall'Asta,,,,,,,,,,http://revistas.utfpr.edu.br/pb/index.php/CAP/...,CAP Accounting and Management
4,2012-01-01,Percepçőes dos profissionais de contabilidade ...,O fluxo contínuo de inovaçőes tecnológicas com...,,4,Ricardo Adriano Antonelli,Lauro Brito de Almeida,Romualdo Douglas Colauto,Fernanda Luiza Longhi,,,,,,,,,http://revistas.utfpr.edu.br/pb/index.php/CAP/...,CAP Accounting and Management


In [23]:
df.to_csv('../data/CAP.csv', index=False)

In [24]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afilição 1,Afilição 2,Afilição 3,Afilição 4,Afilição 5,Afilição 6,Url Artigo,Revista
0,2016-01-01,Análise dos artigos sobre a contabilidade ambi...,A crise ambiental está diretamente vinculada ŕ...,,1,Cleber Broietti,,,,,,,,,,,,http://revistas.utfpr.edu.br/pb/index.php/CAP/...,CAP Accounting and Management
1,2015-01-01,Empresas sustentáveis no Brasil: o que elas te...,Empresas sustentáveis săo aquelas que atuam em...,,2,Cintia Maria Araújo,Leonardo José Seixas Pinto,,,,,,,,,,,http://revistas.utfpr.edu.br/pb/index.php/CAP/...,CAP Accounting and Management
2,2020-01-01,INVESTIGAÇĂO DOS ARTEFATOS GERENCIAIS UTILIZAD...,O presente estudo objetiva investigar os artef...,,5,Rosa Jaqueline Ortiz Hamermüller,Yuri Gomes Paiva Azevedo,Hellen Bomfim Gomes,Raimundo Marciano de Freitas Neto,Aneide Oliveira Araújo,,,,,,,,http://revistas.utfpr.edu.br/pb/index.php/CAP/...,CAP Accounting and Management
3,2017-01-01,CONTABILIDADE DE CUSTOS SOB A ÓTICA DA TEORIA ...,A perspectiva institucional está se tornando u...,,3,Keila Priscila Santos,Juliano Francisco Baldissera,Denis Dall'Asta,,,,,,,,,,http://revistas.utfpr.edu.br/pb/index.php/CAP/...,CAP Accounting and Management
4,2012-01-01,Percepçőes dos profissionais de contabilidade ...,O fluxo contínuo de inovaçőes tecnológicas com...,,4,Ricardo Adriano Antonelli,Lauro Brito de Almeida,Romualdo Douglas Colauto,Fernanda Luiza Longhi,,,,,,,,,http://revistas.utfpr.edu.br/pb/index.php/CAP/...,CAP Accounting and Management
5,2015-01-01,A EVOLUÇĂO DOS RESTOS A PAGAR NĂO PROCESSADOS ...,A inscriçăo dos restos a pagar năo processados...,,5,Ronaldo Cardoso Garcia Filho,Eduardo José Costa Melo,Alex Laquis Resende,Luiz Medeiros de Araujo Neto,Fernanda Ledo Marciniuk,,,,,,,,http://revistas.utfpr.edu.br/pb/index.php/CAP/...,CAP Accounting and Management
6,2019-01-01,APLICAÇĂO DE METODOLOGIA DE CUSTEIO COM BASE N...,Através de estudos sobre os gastos do transpor...,,3,Cleyde Cristina Rodrigues,Ana Clara Fonseca do Amaral,Gilberto Januário Rodrigues,,,,,,,,,,http://revistas.utfpr.edu.br/pb/index.php/CAP/...,CAP Accounting and Management
7,2012-01-01,O reflexo dos ativos intangíveis no valor de m...,No mercado competitivo em que as empresas se e...,,3,Claudio Antonio Rojo,Almir Ferreira de Sousa,Francielle Trento,,,,,,,,,,http://revistas.utfpr.edu.br/pb/index.php/CAP/...,CAP Accounting and Management
8,2016-01-01,Licitaçăo sustentável como promotora de compet...,A licitaçăo é um assunto importante dentro da ...,,4,Déborah Machado de Oliveira,Sabrina Alves dos Santos,Marcelo Roger Meneghatti,Geysler Rógis Flor Bertolini,,,,,,,,,http://revistas.utfpr.edu.br/pb/index.php/CAP/...,CAP Accounting and Management
9,2017-01-01,Simulaçăo de cenários estratégicos em uma empr...,O mercado de empresas fabricantes de placas e ...,,4,Thayse Ana Ferreira,Gabriel Borges Ferreira,Claudio Antonio Rojo,Eveline Favero,,,,,,,,,http://revistas.utfpr.edu.br/pb/index.php/CAP/...,CAP Accounting and Management
